In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas
import contextily as cx
from matplotlib.patches import Patch, Polygon

from scripts.utils import save_fig

%load_ext autoreload
%autoreload 2
font = {'weight' : 'regular',
        'family': 'serif',
        'size'   : 16}
plt.rc('font', **font)
plt.rc('xtick', labelsize='14')
plt.rc('ytick', labelsize='14')
COLOR_PALETTE = np.array(sns.color_palette("colorblind"))[[5,0,2,3]]

In [ ]:
colors = [(1, 0.6, 0), (0.3, 0.6, 0.3), (0.8, 0.1, 0.), (0.3, 0.3, 1)]

In [ ]:
appelations = geopandas.read_file("data/wine_regions/2021-12-22-delim-parcellaire-aoc-shp")

In [ ]:
df_locations = pd.read_excel('data/gathered_from_internet/vineyard_locations.xlsx')
df_locations = df_locations.rename(columns={'region': 'Region'})

In [ ]:
appelations = appelations[(appelations["dt"]=="Bordeaux")&(appelations["categorie"]=="Vin tranquille")]

In [ ]:
appelations["denom"].unique()

In [ ]:
gdf = geopandas.GeoDataFrame(
    df_locations[["Region"]], geometry=geopandas.points_from_xy(df_locations.longitude, df_locations.latitude))
gdf.crs = "epsg:4326"
gdf = gdf.to_crs("epsg:3395")

In [ ]:
medoc = appelations[appelations["denom"].isin(["Médoc"])]
pessac = appelations[appelations["denom"].isin(["Pessac-Léognan"])]
pomerol = appelations[appelations["denom"].isin(["Pomerol"])]
st_emilion = appelations[appelations["denom"].isin(["Saint-Emilion"])]

In [ ]:
pessac = pessac.to_crs(gdf.crs.to_string())
medoc = medoc.to_crs(gdf.crs.to_string())
pomerol = pomerol.to_crs(gdf.crs.to_string())
st_emilion = st_emilion.to_crs(gdf.crs.to_string())

Saint-Émilion (Lon = 44.91, Lat = -0.14), Pauillac (LON = -0.767, LAT = 45.182) and Léognan (Lon : -0.542 / Lat: 44.756)

In [ ]:
weather_points = pd.DataFrame({
    'latitude': [44.91, 45.182, 44.756],
    'longitude': [-0.14, -0.767, -0.542],
})
weather_points_gdf = geopandas.GeoDataFrame(
    geometry=geopandas.points_from_xy(weather_points.longitude, weather_points.latitude))
weather_points_gdf.crs = "epsg:4326"
weather_points_gdf = weather_points_gdf.to_crs("epsg:3395")

##### ! Keep high figsize, as it reduces watermark size !

In [ ]:
list_regions = sorted(list(gdf['Region'].unique()))
dict_colors = {list_regions[i]: COLOR_PALETTE[i] for i in range(len(list_regions))}

fig, ax = plt.subplots(figsize=(12,12))
print(plt.gcf().get_size_inches())
gdf.plot(ax=ax, zorder=2, markersize=100, edgecolor="k", linewidth=1.1, categorical=True, marker='s', c=gdf["Region"].map(dict_colors))
weather_points_gdf.plot(ax=ax, zorder=3, markersize=300, linewidth=2, edgecolor="k", facecolor="w", marker='v')

import matplotlib.lines as mlines
triangle_vertice =ax.plot([], markersize=17, markeredgewidth=2, color="w", markeredgecolor="k", marker='v', label="Weather records")[0]

pessac.plot(ax=ax, facecolor=dict_colors["Pessac"], alpha=0.7)
st_emilion.plot(ax=ax,  facecolor=dict_colors["Saint-Emilion"], alpha=0.7)
pomerol.plot(ax=ax,  facecolor=dict_colors["Pomerol"], alpha=0.7)
medoc.plot(ax=ax, facecolor=dict_colors["Medoc"], alpha=0.7)

plt.xlim((-95000, -4000))
plt.ylim((5540000, 5640000))
plt.axis('off')

legend_handles = [
    Patch(facecolor=dict_colors["Medoc"], label="Médoc"),
    Patch(facecolor=dict_colors["Pomerol"], label="Pomerol"),
    Patch(facecolor=dict_colors["Saint-Emilion"], label="Saint-Emilion"),
    Patch(facecolor=dict_colors["Pessac"], label="Pessac-Léognan"),
    triangle_vertice
]
ax.legend(fontsize=14, handles=legend_handles, loc="upper right", frameon=True)
plt.text(-72000, 5566300, "BORDEAUX", family='serif', fontsize=17, weight="bold")#, bbox=dict(boxstyle='square,pad=0.15', fc='white', ec='k'))

cx.add_basemap(ax, alpha=0.4, zorder=-1, source=cx.providers.Stamen.TonerBackground, crs=gdf.crs.to_string(), zoom=13)

save_fig("views/map", "double")
plt.show()

---
# End of notebook
